## Define Model

In [1]:
import matplotlib
from keras import backend as K
K.set_image_dim_ordering('th')

/home/callum/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
Using cuDNN version 7005 on context None
Preallocating 4861/6077 Mb (0.800000) on cuda0
Mapped name None to device cuda0: GeForce GTX 1060 6GB (0000:01:00.0)


In [2]:
from keras.models import Sequential
from keras.layers import ZeroPadding2D, Conv2D
from keras.layers import MaxPooling2D, BatchNormalization
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np

vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def ConvBlock(layers, filters):
        for i in range(layers):
            model.add(ZeroPadding2D((1, 1)))
            model.add(Conv2D(filters, kernel_size=(3, 3), activation='relu'))  # Keras2
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))
def FCBlock():
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.))
        #model.add(BatchNormalization())

In [3]:
model = Sequential()
model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))
ConvBlock(2, 64)
ConvBlock(2, 128)
ConvBlock(3, 256)
ConvBlock(3, 512)
ConvBlock(3, 512)

model.add(Flatten())
FCBlock()
FCBlock()

for layer in model.layers[:37]: layer.trainable=False

model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
#Import past weights

#fname = 'fastai/models/vgg16.h5'
#model.load_weights(fname)

#for layer in model.layers[:31]: layer.trainable=False
#model.pop()
#model.add(Dense(2, activation='softmax'))
#model.compile(optimizer=Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
#last_conv_idx = [index for index,layer in enumerate(model.layers) 
#                     if type(layer) is Conv2D][-1]
#last_conv_idx

30

In [38]:
conv_layers = model.layers[:31]
conv_model = Sequential(conv_layers)
fc_layers = model.layers[31:]

In [41]:
from keras.regularizers import l2, l1
opt = RMSprop(lr=0.00001, rho=0.7)

# SINCE KERAS MAKES USE OF INVERTED DROPOUT WE "NEUTRALIZE" proc_wgts(layer):
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

def get_fc_model():
    model = Sequential([
        #MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(input_shape=original_model.output_shape[1:]),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(2, activation='softmax')
    ])
    
    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [52]:
model = VGG16(weights='imagenet', 
                           include_top=False,
                            input_shape=(3,224,224))


# CREATE A TOP MODEL
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='sigmoid'))

In [73]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
input = Input(shape=(3,224,224),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(2, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)
my_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 3, None, None)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, None, None)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, None, None)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, None, None)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, None, None)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, None, None)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, None, None)   0         
__________

/home/callum/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=flatten_5/...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 3, 224, 224)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              102764544 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 2)                 8194      
Total params: 134,268,738
Trainable params: 134,268,738
Non-trainable params: 0
______________________________________________________________

In [70]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False
for layer in my_model.layers[:3]:
    model.trainable = False
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 3, 224, 224)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              102764544 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 8)                 32776     


/home/callum/anaconda2/lib/python2.7/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Total params: 149,008,008
Trainable params: 134,293,320
Non-trainable params: 14,714,688
_________________________________________________________________


### Import the data

In [9]:
path='data/dogscats'
train_path=path+"/train"
valid_path=path+"/valid"

In [10]:
#Get data
from keras.preprocessing import image

gen = image.ImageDataGenerator()
batch_size=16

train_batches = gen.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size)
val_batches = gen.flow_from_directory(valid_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=batch_size)

classes = list(iter(train_batches.class_indices)) # get a list of all the class labels

for c in train_batches.class_indices:
    classes[train_batches.class_indices[c]] = c

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [54]:
steps_per_epoch = int(np.ceil(train_batches.samples/batch_size))
validation_steps = int(np.ceil(val_batches.samples/batch_size))

trn_classes = train_batches.classes
val_classes = val_batches.classes
val_features = conv_model.predict_generator(val_batches, validation_steps)
trn_features = conv_model.predict_generator(train_batches, steps_per_epoch)
trn_features.shape

(23000, 512, 14, 14)

In [11]:
import bcolz
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

In [57]:
save_array('models/train_convlayer_features.bc', trn_features)
save_array('models/valid_convlayer_features.bc', val_features)

In [12]:
from keras.utils.np_utils import to_categorical
def onehot(x):
    return to_categorical(x)

In [59]:
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)
save_array('models/train_convlayer_labels.bc', trn_labels)
save_array('models/valid_convlayer_labels.bc', val_labels)

In [13]:
trn_features = load_array('models/train_convlayer_features.bc')
val_features = load_array('models/valid_convlayer_features.bc')
trn_labels = load_array('models/train_convlayer_labels.bc')
val_labels = load_array('models/valid_convlayer_labels.bc')
trn_features.shape

(23000, 512, 14, 14)

## Fit the model

In [15]:
trn_labels[1]

array([1., 0.])

In [20]:
# Fit FC Layers

#fc_model.fit(trn_features, trn_labels, epochs=1, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/1
23000/23000 [==============================] - 53s 2ms/step - loss: 0.5112 - acc: 0.7601 - val_loss: 0.9457 - val_acc: 0.4995


In [71]:
#Fit Model
#model.load_weights('models/dogscats2.h5')
my_model.fit_generator(train_batches, steps_per_epoch=int(np.ceil(train_batches.samples/batch_size)), epochs=1,
                validation_data=val_batches, validation_steps=int(np.ceil(val_batches.samples/batch_size)))
#model.fit_generator(train_batches, samples_per_epoch=train_batches.samples, epochs=1, validation_data=val_batches, validation_steps=val_batches.samples)

Epoch 1/1


ValueError: Error when checking target: expected predictions to have shape (None, 8) but got array with shape (16, 2)

In [49]:
#model.save_weights('models/dogscats2.h5')

## Examine Results

In [ ]:
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
import matplotlib.pyplot as plt
#Use the plots helper function
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')



In [ ]:
sample_batches = gen.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=4)

In [ ]:
# Get a few images and their true labels
imgs, labels = next(sample_batches)
indxs = np.argmax(labels, axis=1)
indxs
labels = [classes[indx] for indx in indxs]
true_labels_dict = {
    'c0': 'safe driving',
    'c1': 'texting - right',
    'c2': 'talking on the phone - right',
    'c3': 'texting - left',
    'c4': 'talking on the phone - left',
    'c5': 'operating the radio',
    'c6': 'drinking',
    'c7': 'reaching behind',
    'c8': 'hair and makeup',
    'c9': 'talking to passenger',
}
true_labels = [true_labels_dict[label] for label in labels]

In [ ]:
plots(imgs, titles=true_labels)

## Test Model

In [ ]:
model.load_weights('models/state_farm.h5')
batches = gen.flow_from_directory(path+'/test', target_size=(224,224), class_mode='categorical', shuffle=True, batch_size=20)
preds = []
for i in range(batches.samples):
    print i, '/', batches.samples
    ims, labs = batches.next()
    pred = model.predict(ims)
    if preds == []: preds = pred
    preds = np.append(preds, pred, axis=0)

In [ ]:
preds